In [ ]:
import AxionAnalysis as AA

from pathlib import Path as p
import h5py
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
exp_name = p.cwd().name
direction = 'down'
fname = p.cwd() / (exp_name+'.hdf5')
f = h5py.File(fname,'r')

exp_title = ''

print(exp_name)

In [ ]:
mag_data21, mag_data11 ,mag_data22 ,mag_data_db21 ,mag_data_db11 ,mag_data_db22 ,freq ,phi ,rad = AA.data_processing(f)

In [ ]:
impedance_filter = '' 
cutoff_freq = 
atten_level =  
tuning_arm_material = ''
tuning_arm_size = 
tuning_rod_type = '' 
rod_finish = ''
cavity_finish = '' 
SMA_pos = '' 
SMA_len =  
torque_vals =

# S21

In [ ]:
AA.mode_map(mag_data_db21, freq, rad, exp_title)

In [ ]:
AA.S21_slider(mag_data_db21, freq, phi)

In [ ]:
AA.S21_plot(mag_data_db21, freq)

In [ ]:
TM010_Q,TM010_f0=AA.Loaded_Q(mag_data_db21, freq, phi, f, exp_title)

In [ ]:
TM010_Qu, s110, s220, sT0 = AA.TM010_Qu(mag_data_db11, mag_data_db22, phi, freq, TM010_Q, TM010_f0, exp_title)

In [ ]:
def create_or_update_dataset(file, dataset_name, data_dict):
    if dataset_name in file:
        dataset_group = file[dataset_name]
    else:
        dataset_group = file.create_group(dataset_name)
    
    for subgroup_name, subgroup_data in data_dict.items():
        subgroup = dataset_group.create_group(subgroup_name)
        for key, value in subgroup_data.items():
            subgroup[key] = value

with h5py.File('/Users/tcaligiure/Documents/Projects/ADMX/ADMX_data.hdf5', 'w') as master_file:
    exp_data = {
        "impedance_filter": {
            "filter_type": impedance_filter,
            "cutoff_frequency": cutoff_freq,
            "attenuation_level": atten_level
        },
        "tuning": {
            "internal_tuning_arm_material": tuning_arm_material,
            "tuning_arm_size": tuning_arm_size,
            "tuning_rod_type": tuning_rod_type,
            "rod_surface_finish": rod_finish,
            "cavity_surface_finish": cavity_finish
        },
        "antenna": {
            "position": SMA_pos,
            "lengths": SMA_len
        },
        "others": {
            "torque_values": torque_vals,
            "frequency_range": TM010_f0,
            "loaded_Q": TM010_Q,
            "unloaded_Q": TM010_Qu
        }
    }

    create_or_update_dataset(master_file, exp_title, exp_data)

In [ ]:
Qu_1, f1_list = AA.unloaded_Q(mag_data_db11, mag_data_db22, phi, freq, TM010_Q, TM010_f0, 1, exp_title)
Qu_2, f2_list = AA.unloaded_Q(mag_data_db11, mag_data_db22, phi, freq, TM010_Q, TM010_f0, 2, exp_title)

In [ ]:
fig,ax = plt.subplots(1,figsize=(13,8), dpi = 300)

plt.scatter(TM010_f0, TM010_Qu, label = 'TM010', color = 'black')
plt.scatter(f1_list[0:-1], Qu_1, label = 'TM011', color = 'green', marker = 's')
plt.scatter(f2_list[0:-1], Qu_2, label = 'TM012', color = 'red', marker = 'x')
plt.xlabel('Freq [GHz]')
plt.ylabel('$Q_u$')
plt.legend(loc = 'best', frameon = False)
plt.title('$Q_u$ for First Three Modes')
plt.savefig('Multimode_Q.PNG', dpi = 300, facecolor = 'w')

plt.show()

In [ ]:
beta1, beta2 = AA.betas(TM010_Qu, TM010_f0, phi, s110, s220, sT0, exp_title)

In [ ]:
Q1, Q2 = AA.Q_params(TM010_Qu, TM010_f0, phi, beta1, beta2, exp_title)

In [ ]:
AA.Q_check(TM010_Q, TM010_f0, TM010_Qu, phi, Q1, Q2, exp_title)

In [ ]:
def func(c1, c2):
    arg = (1/TM010_Q) + (1/(Q1*c1)) + (1/(Q2*c2))
    return (1/arg)

fig, ax = plt.subplots(1, 1, figsize = (13, 8), dpi = 300)

plt.scatter(TM010_f0, func(1, 1), label = 'Measured $Q_L$', color = 'navy')
plt.plot(TM010_f0, func(0.9, 0.9), label = '$Q_L$, dec. $Q_{1,2}$', color = 'royalblue')
plt.plot(TM010_f0, func(1.1, 1.1), label = '$Q_L$, inc. $Q_{1,2}$', color = 'slategrey')
plt.plot(TM010_f0, func(1.1, 0.9), label = '$Q_L$, alt. $Q_{1,2}$', color = 'slateblue')
plt.legend(loc = 'best', frameon = False)

plt.show()

# S11

In [ ]:
AA.S11_plot(mag_data_db11, freq)

In [ ]:
AA.S11_slider(mag_data_db11, freq, phi)

# S22

In [ ]:
AA.S22_plot(mag_data_db22, freq)

In [ ]:
AA.S22_slider(mag_data_db22, freq, phi)